In [3]:
from dotenv import load_dotenv
import os

In [4]:
# Load environment variables.
load_dotenv()

# Set the model name for our LLMs.
GEMINI_MODEL = "gemini-1.5-flash"
# Store the API key in a variable.
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [5]:
import json

with open("kaiser_urls.json", "r") as f:
    urls = json.load(f)

In [6]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.tools import Tool
from langchain.agents import initialize_agent, load_tools
from langchain_google_genai import ChatGoogleGenerativeAI
import gradio as gr

# ✅ Initialize the model
llm = ChatGoogleGenerativeAI(google_api_key=GEMINI_API_KEY, model=GEMINI_MODEL, temperature=0.3)

# ✅ Step 1: Load the webpages
loader = WebBaseLoader(urls)
docs = loader.load()

# ✅ Step 2: Split & embed the content
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
splits = splitter.split_documents(docs)

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
# vectorstore = FAISS.from_documents(splits, embeddings)
vectorstore = FAISS.load_local(
    "kaiser_faiss_index",
    embeddings,
    allow_dangerous_deserialization=True  # ✅ explicit opt-in
)


# ✅ Create the retrieval QA chain
website_qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=vectorstore.as_retriever())

# ✅ Wrap as a custom tool
website_tool = Tool(
    name="WebsiteQA",
    func=lambda q: website_qa_chain.run(q),
    description="Useful for answering questions based on the provided Kaiser Permanente webpages."
)

# ✅ Set up the built-in tools (Wikipedia & OpenWeatherMap)
tools = load_tools(['wikipedia', "openweathermap-api"], llm=llm)

# ✅ Combine the tools
all_tools = tools + [website_tool]

# ✅ Initialize the agent
agent = initialize_agent(
    all_tools,
    llm=llm,
    verbose=False,
    handle_parsing_errors=True,
    max_iterations=10
)

# ✅ Define the smart agent query function for Gradio
def smart_agent_query_gradio(query_text):
    print("\n🔎 Trying WebsiteQA tool first...")
    website_answer = website_qa_chain.run(query_text)

    fallback_needed = False
    if not website_answer or len(website_answer.strip()) < 30:
        fallback_needed = True
    fallback_keywords = ["I don't know", "no relevant information", "unable to find", "sorry"]
    if any(kw in website_answer.lower() for kw in fallback_keywords):
        fallback_needed = True

    if fallback_needed:
        print("\n↩️ Falling back to the agent (Wikipedia/OpenWeatherMap)...")
        result = agent.invoke({"input": query_text})
        return result["output"]
    else:
        return website_answer

# ✅ Build Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("# 🤖 Kaiser Permanente AI Agent\nAsk any question below and I'll find the answer!")

    with gr.Row():
        query_input = gr.Textbox(
            lines=2,
            placeholder="Ask your question here...",
            label="Your Question"
        )
    with gr.Row():
        output = gr.Textbox(
            lines=10,
            label="AI Answer"
        )

    submit_button = gr.Button("Get Answer")

    # Link the function
    submit_button.click(fn=smart_agent_query_gradio, inputs=query_input, outputs=output)

# ✅ Launch Gradio app
demo.launch()


USER_AGENT environment variable not set, consider setting it to identify your requests.
C:\Users\19163\AppData\Local\Temp\ipykernel_9816\3565807022.py:48: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.



🔎 Trying WebsiteQA tool first...


C:\Users\19163\AppData\Local\Temp\ipykernel_9816\3565807022.py:59: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  website_answer = website_qa_chain.run(query_text)



🔎 Trying WebsiteQA tool first...
